# PSET 3

In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import statsmodels.api as sm  

In [2]:
# improt data
file_path = '/Users/veronica/Dropbox/PhD/2024_2/EC_709/PSETS_EC709/PART2_PSET1/ps2-data.xlsx' 
data = pd.ExcelFile(file_path).parse('Sheet1')


In [18]:
# mae time a date
data['time'] = pd.to_datetime(data['time'])

# define all periods
pre_volcker_start = '1960-01-01'
pre_volcker_end = '1979-06-30'
volcker_greenspan_start = '1979-07-01'
volcker_greenspan_end = '1996-12-31'

# laggs and instruments for 4 lags and 2 lags
variables = ['inf_cpi', 'inf_ppi', 'm2g', 'spread', 'r', 'x']
lags_4 = 4
lags_2 = 2

# RR* values
rr_star_pre_volcker = 0.8742997
rr_star_volcker_greenspan = 3.502469



In [19]:
# functions

# make lags
def create_lags(df, variables, lags):
    lagged_data = {}
    for var in variables:
        for lag in range(1, lags + 1):
            lagged_data[f"{var}_lag{lag}"] = df[var].shift(lag)
    return pd.DataFrame(lagged_data)

# gmm objective fcnt
def gmm_objective(params, y, X, Z):
    beta = params
    residuals = y - X @ beta
    moments = Z.T @ residuals
    W = np.eye(Z.shape[1])  # Weighting matrix
    return moments.T @ W @ moments

# estimating fcn
def estimate_gmm(y, X, Z):
    beta_init = np.zeros(X.shape[1])
    result = minimize(gmm_objective, beta_init, args=(y, X, Z), method='BFGS')
    return result

# data processing for gmm - add constants for estimation of z -> do i?
def prepare_data_for_gmm(data, rr_star):
    y = data['r'].values
    X = sm.add_constant(data[['r_lag1', 'r_lag2', 'inf_cpi', 'x']]).values
    X[:, 0] += rr_star  # Adjust constant term
    Z = sm.add_constant(data.drop(columns=['r', 'time', 'inf_cpi', 'inf_ppi', 'm2g', 'spread', 'x']).values)
    return y, X, Z

# CI calculations
def compute_confidence_intervals(result, y, X, Z):
    beta = result.x
    residuals = y - X @ beta
    G = -(Z.T @ X) / len(y)
    S = (Z.T @ np.diag(residuals**2) @ Z) / len(y)
    var_beta = np.linalg.inv(G.T @ G) @ G.T @ S @ G @ np.linalg.inv(G.T @ G) / len(y)
    se = np.sqrt(np.diag(var_beta))
    ci_lower = beta - 1.645 * se
    ci_upper = beta + 1.645 * se
    return pd.DataFrame({
        'Estimate': beta,
        'Std. Error': se,
        '90% CI Lower': ci_lower,
        '90% CI Upper': ci_upper
    })

In [20]:
# split in periods
pre_volcker_data = data[(data['time'] >= pre_volcker_start) & (data['time'] <= pre_volcker_end)]
volcker_greenspan_data = data[(data['time'] >= volcker_greenspan_start) & (data['time'] <= volcker_greenspan_end)]

# make lags for instruments

pre_volcker_lags_4 = create_lags(pre_volcker_data, variables, lags_4).dropna()
pre_volcker_lags_2 = create_lags(pre_volcker_data, variables, lags_2).dropna()
volcker_greenspan_lags_4 = create_lags(volcker_greenspan_data, variables, lags_4).dropna()
volcker_greenspan_lags_2 = create_lags(volcker_greenspan_data, variables, lags_2).dropna()

# merge back lags into original data
pre_volcker_final_4 = pre_volcker_data.iloc[lags_4:].reset_index(drop=True).join(pre_volcker_lags_4.reset_index(drop=True))
pre_volcker_final_2 = pre_volcker_data.iloc[lags_2:].reset_index(drop=True).join(pre_volcker_lags_2.reset_index(drop=True))
volcker_greenspan_final_4 = volcker_greenspan_data.iloc[lags_4:].reset_index(drop=True).join(volcker_greenspan_lags_4.reset_index(drop=True))
volcker_greenspan_final_2 = volcker_greenspan_data.iloc[lags_2:].reset_index(drop=True).join(volcker_greenspan_lags_2.reset_index(drop=True))


# data processing
y_pre_4, X_pre_4, Z_pre_4 = prepare_data_for_gmm(pre_volcker_final_4, rr_star_pre_volcker)
y_pre_2, X_pre_2, Z_pre_2 = prepare_data_for_gmm(pre_volcker_final_2, rr_star_pre_volcker)
y_volcker_4, X_volcker_4, Z_volcker_4 = prepare_data_for_gmm(volcker_greenspan_final_4, rr_star_volcker_greenspan)
y_volcker_2, X_volcker_2, Z_volcker_2 = prepare_data_for_gmm(volcker_greenspan_final_2, rr_star_volcker_greenspan)

# gmm estimation for all models with 4 and 2 lags
gmm_pre_volcker_4 = estimate_gmm(y_pre_4, X_pre_4, Z_pre_4)
gmm_pre_volcker_2 = estimate_gmm(y_pre_2, X_pre_2, Z_pre_2)
gmm_volcker_greenspan_4 = estimate_gmm(y_volcker_4, X_volcker_4, Z_volcker_4)
gmm_volcker_greenspan_2 = estimate_gmm(y_volcker_2, X_volcker_2, Z_volcker_2)

# get CIs
ci_pre_volcker_4 = compute_confidence_intervals(gmm_pre_volcker_4, y_pre_4, X_pre_4, Z_pre_4)
ci_pre_volcker_2 = compute_confidence_intervals(gmm_pre_volcker_2, y_pre_2, X_pre_2, Z_pre_2)
ci_volcker_greenspan_4 = compute_confidence_intervals(gmm_volcker_greenspan_4, y_volcker_4, X_volcker_4, Z_volcker_4)
ci_volcker_greenspan_2 = compute_confidence_intervals(gmm_volcker_greenspan_2, y_volcker_2, X_volcker_2, Z_volcker_2)

In [21]:
# Display results
print("Pre-Volcker Period Estimates (4 Lags)")
print(ci_pre_volcker_4)
print("\nVolcker-Greenspan Period Estimates (4 Lags)")
print(ci_volcker_greenspan_4)

Pre-Volcker Period Estimates (4 Lags)
   Estimate  Std. Error  90% CI Lower  90% CI Upper
0  0.386933    0.197764      0.061611      0.712255
1  1.055082    0.360749      0.461650      1.648513
2 -0.364335    0.314530     -0.881737      0.153067
3  0.208467    0.098192      0.046941      0.369993
4  0.122418    0.060169      0.023441      0.221395

Volcker-Greenspan Period Estimates (4 Lags)
   Estimate  Std. Error  90% CI Lower  90% CI Upper
0  0.176437    0.171282     -0.105321      0.458196
1  0.325994    0.146360      0.085232      0.566755
2  0.340211    0.153971      0.086928      0.593494
3  0.438404    0.353030     -0.142330      1.019138
4  0.017386    0.139074     -0.211390      0.246162


In [22]:
print("\nPre-Volcker Period Estimates (2 Lags)")
print(ci_pre_volcker_2)
print("\nVolcker-Greenspan Period Estimates (2 Lags)")
print(ci_volcker_greenspan_2)


Pre-Volcker Period Estimates (2 Lags)
   Estimate  Std. Error  90% CI Lower  90% CI Upper
0  0.351011    0.181725      0.052074      0.649949
1  0.830655    0.410563      0.155279      1.506030
2 -0.194103    0.367585     -0.798780      0.410574
3  0.280683    0.105563      0.107032      0.454335
4  0.157364    0.075082      0.033855      0.280874

Volcker-Greenspan Period Estimates (2 Lags)
   Estimate  Std. Error  90% CI Lower  90% CI Upper
0  0.262996    0.143597      0.026779      0.499214
1  0.323205    0.127030      0.114241      0.532169
2  0.327865    0.139905      0.097721      0.558009
3  0.413708    0.199543      0.085460      0.741956
4  0.085989    0.114476     -0.102324      0.274302
